# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = 'output_data/weather_data.csv'

weather_df = pd.read_csv(file_path)
weather_df

,Unnamed: 0,city,max_temp,humidity,wind_speed,cloudiness,lat,lng,country,date
0,0,Cherskiy,46.33,59,4.68,71,68.7500,161.3000,RU,1629149606
1,1,Barrow,37.42,75,23.02,90,71.2906,-156.7887,US,1629149606
2,2,Hobart,46.26,68,3.00,75,-42.8794,147.3294,AU,1629149405
3,3,Kapaa,84.52,67,5.01,1,22.0752,-159.3190,US,1629149607
4,4,Busselton,51.08,71,5.06,9,-33.6500,115.3333,AU,1629149607
...,...,...,...,...,...,...,...,...,...,...
524,524,Abomey,73.15,85,7.20,100,7.1829,1.9912,BJ,1629149757
525,525,Qasigiannguit,46.65,91,5.08,100,68.8193,-51.1922,GL,1629149757
526,526,China,98.64,29,5.99,2,25.7000,-99.2333,MX,1629149678
527,527,Balabac,81.34,75,4.27,66,7.9868,117.0645,PH,1629149757


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store lat and lng into locations
locations = weather_df[['lat', 'lng']].astype(float)

In [5]:
humidity = weather_df['humidity'].astype(float)
weather_data_clean = weather_df.dropna()

In [6]:
# create a humidty layer heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False,
                                 max_intensity=300, point_radius=5)

figure_layout = {
    'width': '1000px',
    'height':'1000px',
}

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# create df with max temp between 70 and 80 degrees, wind speed is less than 10 mph and 0 cloudiness
hotel_df = weather_data_clean.loc[(weather_data_clean['max_temp'] < 80) & (weather_data_clean['max_temp'] > 70)\
                                    & (weather_data_clean['wind_speed'] < 10) & (weather_data_clean['cloudiness'] == 0)]

hotel_df

,Unnamed: 0,city,max_temp,humidity,wind_speed,cloudiness,lat,lng,country,date
13,13,Presidencia Roque Sáenz Peña,75.04,36,5.88,0,-26.7852,-60.4388,AR,1629149609
62,62,Potamiá,78.66,59,7.09,0,40.7163,24.7286,GR,1629149623
181,181,Pirot,74.93,32,5.35,0,43.1531,22.5861,RS,1629149656
196,196,Salta,77.59,25,5.99,0,-24.7859,-65.4117,AR,1629149660
334,334,Suez,77.88,67,9.86,0,29.9737,32.5263,EG,1629149699


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
target_coordinates

(13    -26.7852
 62     40.7163
 181    43.1531
 196   -24.7859
 334    29.9737
 Name: lat, dtype: float64,
 13    -60.4388
 62     24.7286
 181    22.5861
 196   -65.4117
 334    32.5263
 Name: lng, dtype: float64)

In [8]:

# create loop to go through each hotel and pull the first result
for index, row in hotel_df.iterrows():
    hotel_lat = row['lat']
    hotel_lng = row['lng']
    target_coordinates = f"{hotel_lat},{hotel_lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"
    
    # set up parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    # complete api pull
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    hotel_result = response['results'][0]['name']
    print(f"Attempting search at {target_coordinates}.")
    print(f"The name of the hotel near that location is {hotel_result}.")
    hotel_df.loc[index, 'Hotel Name'] = hotel_result

Attempting search at -26.7852,-60.4388.
The name of the hotel near that location is Hotel Gualok.


C:\Users\nicho\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\nicho\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Attempting search at 40.7163,24.7286.
The name of the hotel near that location is Alexandra Golden Thassos Boutique Hotel.
Attempting search at 43.1531,22.5861.
The name of the hotel near that location is Hotel Dijana.
Attempting search at -24.7859,-65.4117.
The name of the hotel near that location is Design Suites Salta.
Attempting search at 29.9737,32.5263.
The name of the hotel near that location is Green House.


In [9]:
hotel_df

,Unnamed: 0,city,max_temp,humidity,wind_speed,cloudiness,lat,lng,country,date,Hotel Name
13,13,Presidencia Roque Sáenz Peña,75.04,36,5.88,0,-26.7852,-60.4388,AR,1629149609,Hotel Gualok
62,62,Potamiá,78.66,59,7.09,0,40.7163,24.7286,GR,1629149623,Alexandra Golden Thassos Boutique Hotel
181,181,Pirot,74.93,32,5.35,0,43.1531,22.5861,RS,1629149656,Hotel Dijana
196,196,Salta,77.59,25,5.99,0,-24.7859,-65.4117,AR,1629149660,Design Suites Salta
334,334,Suez,77.88,67,9.86,0,29.9737,32.5263,EG,1629149699,Green House


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

['\n<dl>\n<dt>Name</dt><dd>13                                Hotel Gualok\n62     Alexandra Golden Thassos Boutique Hotel\n181                               Hotel Dijana\n196                        Design Suites Salta\n334                                Green House\nName: Hotel Name, dtype: object</dd>\n<dt>City</dt><dd13     Presidencia Roque Sáenz Peña\n62                          Potamiá\n181                           Pirot\n196                           Salta\n334                            Suez\nName: city, dtype: object</dd>\n<dt>Country</dt><dd>13     AR\n62     GR\n181    RS\n196    AR\n334    EG\nName: country, dtype: object</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>13                                Hotel Gualok\n62     Alexandra Golden Thassos Boutique Hotel\n181                               Hotel Dijana\n196                        Design Suites Salta\n334                                Green House\nName: Hotel Name, dtype: object</dd>\n<dt>City</dt><dd13     Presidencia Ro

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, 
                            info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))